In [1]:
# 네이버 검색 API 예제 - 블로그 검색
# 출처: https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EB%B8%94%EB%A1%9C%EA%B7%B8

import time
import os
import sys
import json
import re
import urllib.request
import openai
import random
from utils import generate_text_davinci, generate_text_GPT2, NaverSearchAPI, load_embed_model

client_id = ""
client_secret = ""
openai.api_key = "sk-"# **GPT  key 지정

naver_api = NaverSearchAPI(client_id=client_id, client_secret=client_secret)

# 임베딩 모델 로딩
#WORD_EMBDDING_MODEL1, BI_ENCODER1 = load_embed_model('./model/kpf-sbert-v1.1', 'mean', 768, "cpu")


In [14]:
start_time = time.time()

query = "제주도 내일 날씨"  
#contexts, error = naver_api.search_naver_ex(query=query, classification=['news','webkr'], display=10, top_k=10, bi_encoder=BI_ENCODER1)

# 네이버 검색
classification=['webkr', 'blog', 'news']

 # 랜덤하게 2개 선택
selected_items = random.sample(classification, 2)
#random.shuffle(classification)
start = random.randint(1, 3)

print(f'classification:{selected_items}/start:{start}')

contexts, error = naver_api.search_naver(query=query, classification=classification, 
                                         start=start, display=8)
        
# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}\n')

classification:['webkr', 'blog']/start:1
time:0.57



In [15]:
context:str = ""
text:str = ""
for con in contexts:
    context += con['descript']+'\n\n'
    text += con['title']+'\n'+ con['descript'] + '\n'+con['link']+'\n('+con['score']+')\n\n'
print(text)

제주시, 제주도, 대한민국 3일 날씨 예보 | AccuWeather
제주시, 제주도, 대한민국 Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.
https://www.accuweather.com/ko/kr/jeju/224209/weather-forecast/224209
(0)

날씨제주시-정확하고 상세 일기 예보제주시를 위한 오늘날,내일 주습니다. 제주시, 제주특별자치도, 대한민국
MeteoTrend:날씨제주시를 위한 오늘날,내일 주습니다. 정확하고 상세 일기 예보제주시니다. 온도와 습도,공기의 압력,속도,바람의 방향,강수량,바니다. 제주시, 제주특별자치도, 대한민국
https://ko.meteotrend.com/forecast/kr/jeju/
(0)

제주지방 기상청
기상정보, 날씨예보, 계절별 특이사항, 공항날씨 등 제공.
https://www.kma.go.kr/jeju/html/main/index.jsp
(0)

[날씨해설] [오늘날씨] 오늘 제주도·남부지방 비, 내일 전국 비. 4월 14일 5시 기준
기상청 날씨예보는 출·퇴근길 기상정보 및 기상이슈에 대한 원인과 배경을 전문적으로 설명하는 영상입니다. 매일 7시와 17시에 정기방송이 제공되며, 태풍 및 위험기상 발생 등으로 신속한 정보 제공이 필요할 시 비정기 방송이 제공됩니다.
https://www.kma.go.kr/plus/land/movie/weatheron.jsp?num=3104
(0)

날씨 제주 내일 | freemeteo.kr
날씨 제주 내일. 제주, 내일 날씨 보기. 상세 기상 예보. 악천후 알림. freemeteo.kr
https://freemeteo.kr/weather/Jeju-/daily-forecast/tomorrow/?gid=9039766&language=korean&country=south-korea
(0)

[날씨해설] [내일날씨] 내일 오후부터 

In [16]:
start_time = time.time()

prompt = f'{context}\n\nQ:{query}?.요약해줘\nA:'
print(prompt)    
    
response = generate_text_GPT2(gpt_model="gpt-3.5-turbo-1106",
                              prompt=prompt, timeout=10, stream=True)

# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}\n')

print(f'답변:{response[0]}')

제주시, 제주도, 대한민국 Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.

MeteoTrend:날씨제주시를 위한 오늘날,내일 주습니다. 정확하고 상세 일기 예보제주시니다. 온도와 습도,공기의 압력,속도,바람의 방향,강수량,바니다. 제주시, 제주특별자치도, 대한민국

기상정보, 날씨예보, 계절별 특이사항, 공항날씨 등 제공.

기상청 날씨예보는 출·퇴근길 기상정보 및 기상이슈에 대한 원인과 배경을 전문적으로 설명하는 영상입니다. 매일 7시와 17시에 정기방송이 제공되며, 태풍 및 위험기상 발생 등으로 신속한 정보 제공이 필요할 시 비정기 방송이 제공됩니다.

날씨 제주 내일. 제주, 내일 날씨 보기. 상세 기상 예보. 악천후 알림. freemeteo.kr

[날씨해설] [내일날씨] 전국 곳곳 비 또는 눈, 우산 준비하세요! 11월 26일 17시 기준 ; [날씨해설] [오늘날씨] 아침기온 영하권, 대기 건조 산불 주의! 11월 26일 5시 기준 ; [날씨해설] [내일날씨] 내일 아침 전국 대부분 영하권 추위, 동해안을 중심으로 건조해요! 11월 25일 17시 기준 ; [날씨해설] [오늘날씨] 내일 아침까지 추워요, 외투 챙기세요! 11월 25일 5시 기준

- 2023-11-19 21:00:00 o 강풍주의보 : 울릉도독도 o 풍랑주의보 : 동해남부남쪽안쪽먼바다, 동해남부남쪽바깥먼바다, 동해남부북쪽안쪽먼바다, 동해남부북쪽바깥먼바다, 동해중부안쪽먼바다, 동해중부바깥먼바다 o 건조주의보 : 강원도(강릉평지, 동해평지, 삼척평지, 속초평지, 고성평지, 양양평지), 경상북도(울진평지)

국립해양조사원에서 전해드리는 내일의 바다낚시지수입니다. 날씨와 바다의 변화가 들쑥날쑥합니다. 제주도와 남해안에 비 소식 있고요. 서해안 남부 남쪽 먼바다는 금요일까지, 제주도 남쪽 먼바다는... 

새해 복 많이 받으십시오. #오늘의날씨 #내일날씨 #오늘